# Create a table with SRA IDs, readstats QC data, and submitter metadata

**Below are the steps taken in this notebook:**
1. Installs, import Statements & Global Variable Definitions
2. Read in tables as dataframes
3. Merge dataframes
4. Write merged dataframe to data table and file


## Installs

In [1]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [2]:
%%capture
%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [3]:
from firecloud import fiss
import pandas as pd      
import os                 
import subprocess       
import re                 
import io
import gcsfs

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT
from terra_pandas import dataframe_to_table, table_to_dataframe

## Global Variable Declarations

In [4]:
# AnVIL_HPRC WorkspaceBucket
anvil_hprc_bucket       = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"

# table filenames (expected in the workspace bucket, see below)
submitter_metadata = 'UW_HPRC_HiFi_Y3_metadata.txt'
sra_metadata = 'metadata-12971042-processed-ok.tsv'

# submission id
submission_id = 'UW_HPRC_HiFi_Y3'

# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_WRANGLING_UW_HPRC_HiFi_Y3
Workspace storage bucket: gs://fc-c2424fa0-8b70-4a0a-bd92-598d9fa71260/


# Read In tables as dataframes

## Readstats table
Find the individual readstats output files in the table and concatenate their contents.
Add the hifi filename and sample name.

In [55]:
# readstats
readstats_df = table_to_dataframe("readstats", workspace=WORKSPACE, workspace_namespace=PROJECT)
qc_list = []

for index, row in readstats_df.iterrows():

        sample_readstats_fp = row['ReadStatsReport']
        sample_readstats_fn = os.path.basename(sample_readstats_fp)
        df = pd.read_csv(sample_readstats_fp, header=None, sep='\t')
        df = df[df[0]=='sample.fastq']
        # add sample and file name - note it's 5mc filename here
        df = df.append({0:'undef', 1:'filename', 2:os.path.basename(row['output_5mc_bam'][0])}, ignore_index=True)
        # remove all but the variables (total_bp, quartile_25 etc)
        df2 = df.drop(df.columns[[0, 1]], axis=1)
        # make rownames ('total_bp', 'quartile_25' etc)
        df2.index = df.iloc[:,1]
        # remove the now meaningless 'sample.fastq' filename
        df2 = df2.drop(index=['file'])
        qc_list.append(df2)

# merge
readstats_df = pd.concat(qc_list, axis=1).transpose()
readstats_df.head()

2023-03-21 07:13:59::WARNING  Retrying (Retry(total=9, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))': /api/workspaces/human-pangenome-ucsc/HPRC_WRANGLING_UW_HPRC_HiFi_Y3/entityQuery/readstats?page=1&pageSize=500&sortDirection=asc


1,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,quartile_50,quartile_75,N25,N50,N75,filename
2,2287312,47677590021,47.68,87,64390,20844,17755,20124,23333,18329,20991,24424,m54329U_220825_174247-bc2012.5mc.hifi_reads.bam
2,2276772,45528316868,45.53,52,54052,19996,17274,19325,22189,17720,20026,23102,m54329U_220827_143814-bc2050.5mc.hifi_reads.bam
2,1525960,30157424611,30.16,79,64002,19762,15750,18909,22999,16931,20504,24966,m54329U_211222_104516-bc1010.5mc.hifi_reads.bam
2,1643577,32778391595,32.78,61,56516,19943,15883,19084,23193,17062,20664,25118,m54329U_211230_014258-bc1010.5mc.hifi_reads.bam
2,1862777,37535814568,37.54,118,61767,20150,15989,19255,23451,17196,20876,25416,m54329U_211231_123903-bc1010.5mc.hifi_reads.bam


## Submitter metadata table

In [56]:
# remember that we did a Primrose run so the filenames should be 5mc.hifi for matching
meta_fp = os.path.join(bucket, submitter_metadata)
metadata_df = pd.read_csv(meta_fp, sep='\t')
metadata_df = metadata_df[['filename', 'sample_ID', 'instrument_model', 'shear_method', 'size_selection', 'ccs_algorithm', 
    'polymerase_version', 'seq_plate_chemistry_version', 'generator_facility', 'generator_contact']]
#metadata_df[['sample', 'filename']] = metadata_df['filename'].str.split('/', expand=True)
metadata_df['filename'] = metadata_df['filename'].str.replace(r'.*/','', regex=True)
metadata_df['filename'] = metadata_df['filename'].str.replace('hifi_read', '5mc.hifi_read')
metadata_df.head()

,filename,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m54329U_220825_174247-bc2012.5mc.hifi_reads.bam,HG00099,PacBio Sequel II,Megaruptor 3,PippinHT,6.3.0,P3.2,C2,University of Washington,kmiyamot@uw.edu
1,m54329U_220827_143814-bc2050.5mc.hifi_reads.bam,HG00099,PacBio Sequel II,Megaruptor 3,PippinHT,6.3.0,P3.2,C2,University of Washington,kmiyamot@uw.edu
2,m54329U_220829_095708-bc2050.5mc.hifi_reads.bam,HG00099,PacBio Sequel II,Megaruptor 3,PippinHT,6.3.0,P3.2,C2,University of Washington,kmiyamot@uw.edu
3,m54329U_220901_221341-bc2051.5mc.hifi_reads.bam,HG00280,PacBio Sequel II,Megaruptor 3,PippinHT,6.3.0,P3.2,C2,University of Washington,kmiyamot@uw.edu
4,m64076_220831_191646-bc2051.5mc.hifi_reads.bam,HG00280,PacBio Sequel II,Megaruptor 3,PippinHT,6.3.0,P3.2,C2,University of Washington,kmiyamot@uw.edu


## SRA table
Must download after finishing submission.
The downloaded table needs to be split in one row per file (instead of filename, filename2 <...> filenameN)

In [57]:
## Read in SRA file and split by filename
# because of the split biosample/SRA upload, this file has biosample_accessions instead of sample_names

sra_file = os.path.join(bucket + sra_metadata)
sample_df = pd.read_csv(sra_file, sep='\t')
sample_df[['sample_name','ignoreme']] = sample_df['library_ID'].str.split('.',expand=True)
sample_df['sample_name'] = sample_df['sample_name'].str.replace(r'_.*', '', regex=True)

fnames = [x for x in sample_df.columns if x.startswith('filename')]
sra_df = pd.melt(sample_df, id_vars =['sample_name', 'accession'], value_vars = fnames, value_name='file')
sra_df= sra_df.dropna()
sra_df = sra_df[['file', 'accession', 'sample_name']]
sra_df.columns = ['filename', 'accession', 'sample']
sra_df.head()


,filename,accession,sample
0,m54329U_220825_174247-bc2012.5mc.hifi_reads.bam,SRR23922670,PG00099
1,m54329U_220827_143814-bc2050.5mc.hifi_reads.bam,SRR23922669,PG00099
2,m54329U_220901_221341-bc2051.5mc.hifi_reads.bam,SRR23922658,PG00280
3,m54329U_220107_233847-bc1016.5mc.hifi_reads.bam,SRR23922640,HG00558
4,m54329U_211222_104516-bc1010.5mc.hifi_reads.bam,SRR23922643,HG00639


# Merge all dataframes

In [58]:
print(sra_df.shape, readstats_df.shape, metadata_df.shape)
# two samples of HG02572 were not uploaded to SRA with the rest of this batch

(91, 3) (91, 13) (91, 10)


In [59]:
merged_df = pd.merge(
    sra_df,
    readstats_df,
    on='filename')
merged_df = pd.merge(
    merged_df,
    metadata_df,
    on='filename')
merged_df.head()

,filename,accession,sample,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,...,N75,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m54329U_220825_174247-bc2012.5mc.hifi_reads.bam,SRR23922670,PG00099,2287312,47677590021,47.68,87,64390,20844,17755,...,24424,HG00099,PacBio Sequel II,Megaruptor 3,PippinHT,6.3.0,P3.2,C2,University of Washington,kmiyamot@uw.edu
1,m54329U_220827_143814-bc2050.5mc.hifi_reads.bam,SRR23922669,PG00099,2276772,45528316868,45.53,52,54052,19996,17274,...,23102,HG00099,PacBio Sequel II,Megaruptor 3,PippinHT,6.3.0,P3.2,C2,University of Washington,kmiyamot@uw.edu
2,m54329U_220901_221341-bc2051.5mc.hifi_reads.bam,SRR23922658,PG00280,2371461,47176915770,47.18,87,57069,19893,16982,...,23180,HG00280,PacBio Sequel II,Megaruptor 3,PippinHT,6.3.0,P3.2,C2,University of Washington,kmiyamot@uw.edu
3,m54329U_220107_233847-bc1016.5mc.hifi_reads.bam,SRR23922640,HG00558,1605959,33821381595,33.82,61,60416,21059,16632,...,26503,HG00558,PacBio Sequel II,Megaruptor 3,PippinHT,6.2.0,P2.2,C2,University of Washington,kmiyamot@uw.edu
4,m54329U_211222_104516-bc1010.5mc.hifi_reads.bam,SRR23922643,HG00639,1525960,30157424611,30.16,79,64002,19762,15750,...,24966,HG00639,PacBio Sequel II,Megaruptor 3,PippinHT,6.2.0,P2.2,C2,University of Washington,kmiyamot@uw.edu


# Upload To Tables

In [60]:
## Create table
#dataframe_to_table("post_sra_metadata", merged_df, WORKSPACE, PROJECT)
outf = os.path.join(bucket, submission_id + '_post_sra_metadata.tsv')
merged_df.to_csv(outf, sep="\t", index=False)